# Setup

In [1]:
import numpy as np
import pandas as pd
import torch
import os
import re
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

# Tokenize words and get embeddings
def get_word_embeddings(word_list):
    tokens = tokenizer(word_list, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**tokens)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Average pooling over tokens
    return embeddings

# Word Sets and Embeddings
AF_Names = ["Reginald", "Kameron", "Kendrick", "Javon", "Tyrell", "Jamar", "Camron", "Tyree", "Jamari", "Reggie", "Jada", 
            "Latoya", "Jayla", "Tamika", "Latoyna", "Journey", "Tameka", "Journee", "Lawanda", "Janiya"]
AF_Embeddings = get_word_embeddings(AF_Names)

EU_Names = ["James", "John", "Robert", "Michael", "William", "David", "Joseph", "Richard", "Charles", "Thomas", "Mary", 
            "Elizabeth", "Patricia", "Jennifer", "Linda", "Barbara", "Margaret", "Susan", "Sarah", "Jessica"]
EU_Embeddings = get_word_embeddings(EU_Names)

LX_Names = ["Paul", "Vincent", "Victor", "Adrian", "Marcus", "Leo", "Miles", "Roman", "Sergio", "Felix", "Patricia", "Laura", 
            "Amanda", "Victoria", "Julia", "Gloria", "Diana", "Clara", "Paula", "Norma"]
LX_Embeddings = get_word_embeddings(LX_Names)

CH_Names = ["Lian", "Shan", "Lew", "Long", "Quan", "Jun", "Tou", "Jin", "Cai", "Chan", "Lue", "China", "Lu", "Maylee", 
            "Tennie", "Maylin", "Chynna", "Jia", "Mei", "Tylee"]
CH_Embeddings = get_word_embeddings(CH_Names)

Male_Names = ["James", "John", "Robert", "Michael", "William", "David", "Joseph", "Richard", "Charles", "Thomas", 
              "Christopher", "Daniel", "Matthew","George", "Anthony", "Donald", "Paul", "Mark", "Andrew", "Edward"]
Male_Embeddings = get_word_embeddings(Male_Names)

Female_Names = ["Mary", "Elizabeth", "Patricia", "Jennifer", "Linda", "Barbara", "Margaret", "Susan", "Dorothy", "Sarah", 
                "Jessica", "Helen", "Nancy", "Betty", "Karen", "Lisa", "Anna", "Sandra", "Emily", "Ashley"]
Female_Embeddings = get_word_embeddings(Female_Names)

Pleasant_Words = ["happy", "agreeable", "polite", "civil", "charming", "gracious", "gentle", "approachable", "love", "cool"]
Pleasant_Embeddings = get_word_embeddings(Pleasant_Words)

Unpleasant_Words = ["rude", "lazy", "disagreeable", "lousy", "sad", "hate", "violent", "bitter", "harsh", "angry"]
Unpleasant_Embeddings = get_word_embeddings(Unpleasant_Words)

STEM_Careers = ["Software Developer", "Nurse Practitioner", "Health Services Manager", "Physicians Assistant", 
                "Security Analyst", "IT Manager", "Web Developer", "Dentist", "Orthodontist", "Computer Systems Analyst"]
STEM_Embeddings = get_word_embeddings(STEM_Careers)

Non_STEM_Careers = ["Artist", "Marketing Manager", "Social Worker", "Attorney", "Journalist", "Musician", "Teacher", 
                    "Media Manager", "Graphic Designer", "Judge"]
Non_STEM_Embeddings = get_word_embeddings(Non_STEM_Careers)

C:\Users\Ryan Ackerman\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Ryan Ackerman\anaconda3\envs\venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ryan Ackerman\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer

# TEST 1: Racial Biases

In [2]:
# African American Names
# Pleasant Words
similarities_AFvP = cosine_similarity(AF_Embeddings, Pleasant_Embeddings)
# Unpleasant Words
similarities_AFvU = cosine_similarity(AF_Embeddings, Unpleasant_Embeddings)

similarities_AFvP = pd.DataFrame(similarities_AFvP, index = AF_Names, columns = Pleasant_Words)
similarities_AFvU = pd.DataFrame(similarities_AFvU, index = AF_Names, columns = Unpleasant_Words)

print("Cosine Similarity Matrix: African American Names vs Pleasant Words")
print(similarities_AFvP)
print("Cosine Similarity Matrix: African American Names vs Unpleasant Words")
print(similarities_AFvU)

Cosine Similarity Matrix: African American Names vs Pleasant Words
             happy  agreeable    polite     civil  charming  gracious  \
Reginald  0.738282   0.649878  0.777401  0.563782  0.838198  0.532421   
Kameron   0.604671   0.563004  0.624187  0.484266  0.665703  0.517958   
Kendrick  0.620416   0.559042  0.658218  0.557435  0.689143  0.470296   
Javon     0.568455   0.541190  0.564188  0.409593  0.643538  0.515993   
Tyrell    0.590594   0.526279  0.598666  0.526088  0.626096  0.494070   
Jamar     0.717939   0.623204  0.696628  0.555890  0.745717  0.518933   
Camron    0.657159   0.589707  0.662114  0.544168  0.687121  0.485016   
Tyree     0.642630   0.610589  0.636554  0.475978  0.709992  0.516491   
Jamari    0.700243   0.603156  0.667514  0.542031  0.721779  0.558601   
Reggie    0.767005   0.629360  0.763899  0.556539  0.816957  0.506053   
Jada      0.608464   0.537163  0.575162  0.446533  0.606839  0.485288   
Latoya    0.448220   0.507939  0.428840  0.360260  0.4555

In [3]:
# European American Names
# Pleasant Words
similarities_EUvP = cosine_similarity(EU_Embeddings, Pleasant_Embeddings)
# Unpleasant Words
similarities_EUvU = cosine_similarity(EU_Embeddings, Unpleasant_Embeddings)

similarities_EUvP = pd.DataFrame(similarities_EUvP, index = EU_Names, columns = Pleasant_Words)
similarities_EUvU = pd.DataFrame(similarities_EUvU, index = EU_Names, columns = Unpleasant_Words)

print("Cosine Similarity Matrix: European American Names vs Pleasant Words")
print(similarities_EUvP)
print("Cosine Similarity Matrix: European American Names vs Unpleasant Words")
print(similarities_EUvU)

Cosine Similarity Matrix: European American Names vs Pleasant Words
              happy  agreeable    polite     civil  charming  gracious  \
James      0.745801   0.660998  0.744565  0.615661  0.805770  0.596715   
John       0.701383   0.660229  0.715240  0.656073  0.754625  0.578608   
Robert     0.736228   0.661684  0.733061  0.623935  0.792235  0.577117   
Michael    0.778046   0.665122  0.757903  0.621934  0.830386  0.591476   
William    0.715868   0.670677  0.736374  0.618859  0.788804  0.584382   
David      0.748434   0.640911  0.721791  0.596564  0.788930  0.576498   
Joseph     0.739581   0.671847  0.719362  0.592936  0.786319  0.582245   
Richard    0.730502   0.664337  0.725620  0.630241  0.779778  0.587989   
Charles    0.717849   0.691632  0.729507  0.648721  0.775796  0.581864   
Thomas     0.741932   0.663854  0.741360  0.625552  0.804401  0.576369   
Mary       0.761429   0.654816  0.748546  0.597887  0.818544  0.638292   
Elizabeth  0.703669   0.649814  0.714691  0.

In [4]:
# Latin American Names
# Pleasant Words
similarities_LXvP = cosine_similarity(LX_Embeddings, Pleasant_Embeddings)
# Unpleasant Words
similarities_LXvU = cosine_similarity(LX_Embeddings, Unpleasant_Embeddings)

similarities_LXvP = pd.DataFrame(similarities_LXvP, index = LX_Names, columns = Pleasant_Words)
similarities_LXvU = pd.DataFrame(similarities_LXvU, index = LX_Names, columns = Unpleasant_Words)

print("Cosine Similarity Matrix: Latin American Names vs Pleasant Words")
print(similarities_LXvP)
print("Cosine Similarity Matrix: Latin American Names vs Unpleasant Words")
print(similarities_LXvU)

Cosine Similarity Matrix: Latin American Names vs Pleasant Words
             happy  agreeable    polite     civil  charming  gracious  \
Paul      0.748556   0.670138  0.743731  0.614286  0.808490  0.579526   
Vincent   0.763479   0.678518  0.741270  0.614821  0.799807  0.579912   
Victor    0.772577   0.669798  0.733051  0.614408  0.821395  0.604414   
Adrian    0.754619   0.661476  0.744551  0.586766  0.826797  0.582458   
Marcus    0.753258   0.642394  0.719776  0.567663  0.805800  0.583674   
Leo       0.764588   0.680265  0.738488  0.615970  0.806728  0.592112   
Miles     0.752611   0.645188  0.719920  0.552470  0.811447  0.580947   
Roman     0.694632   0.636521  0.726212  0.676198  0.708834  0.576997   
Sergio    0.757843   0.664255  0.719615  0.588998  0.782531  0.570943   
Felix     0.785083   0.664356  0.750478  0.592634  0.846167  0.603942   
Patricia  0.745577   0.687538  0.763339  0.640642  0.800270  0.603197   
Laura     0.788697   0.659056  0.758021  0.595310  0.831445

In [5]:
# Chinese American Names
# Pleasant Words
similarities_CHvP = cosine_similarity(CH_Embeddings, Pleasant_Embeddings)
# Unpleasant Words
similarities_CHvU = cosine_similarity(CH_Embeddings, Unpleasant_Embeddings)

similarities_CHvP = pd.DataFrame(similarities_CHvP, index = CH_Names, columns = Pleasant_Words)
similarities_CHvU = pd.DataFrame(similarities_CHvU, index = CH_Names, columns = Unpleasant_Words)

print("Cosine Similarity Matrix: Chinese American Names vs Pleasant Words")
print(similarities_CHvP)
print("Cosine Similarity Matrix: Chinese American Names vs Unpleasant Words")
print(similarities_CHvU)

Cosine Similarity Matrix: Chinese American Names vs Pleasant Words
           happy  agreeable    polite     civil  charming  gracious    gentle  \
Lian    0.637302   0.593035  0.625992  0.483811  0.670235  0.474502  0.663205   
Shan    0.732074   0.617100  0.753050  0.563863  0.800331  0.492043  0.784405   
Lew     0.692608   0.608179  0.731156  0.618591  0.744155  0.483469  0.750529   
Long    0.677847   0.619128  0.732059  0.531671  0.734361  0.469223  0.760287   
Quan    0.675814   0.603027  0.698052  0.583944  0.706869  0.434508  0.704077   
Jun     0.602203   0.523018  0.622013  0.509027  0.609947  0.386542  0.621506   
Tou     0.675757   0.610610  0.644830  0.482018  0.679285  0.507924  0.689427   
Jin     0.702688   0.621965  0.688770  0.526754  0.742340  0.471629  0.723426   
Cai     0.667200   0.608120  0.703372  0.600657  0.704862  0.461392  0.701819   
Chan    0.730272   0.607904  0.729390  0.542922  0.795287  0.477988  0.777494   
Lue     0.624381   0.602227  0.606552  0.4

# TEST 2: Gender Biases for Favorability

In [6]:
# Male Names
# Pleasant Words
similarities_MvP = cosine_similarity(Male_Embeddings, Pleasant_Embeddings)
# Unpleasant Words
similarities_MvU = cosine_similarity(Male_Embeddings, Unpleasant_Embeddings)

similarities_MvP = pd.DataFrame(similarities_MvP, index = Male_Names, columns = Pleasant_Words)
similarities_MvU = pd.DataFrame(similarities_MvU, index = Male_Names, columns = Unpleasant_Words)

print("Cosine Similarity Matrix: Male Names vs Pleasant Words")
print(similarities_MvP)
print("Cosine Similarity Matrix: Male Names vs Unpleasant Words")
print(similarities_MvU)

Cosine Similarity Matrix: Male Names vs Pleasant Words
                happy  agreeable    polite     civil  charming  gracious  \
James        0.745801   0.660998  0.744565  0.615661  0.805770  0.596715   
John         0.701383   0.660229  0.715240  0.656073  0.754625  0.578608   
Robert       0.736228   0.661684  0.733061  0.623935  0.792235  0.577117   
Michael      0.778046   0.665122  0.757903  0.621934  0.830386  0.591476   
William      0.715868   0.670677  0.736374  0.618859  0.788804  0.584382   
David        0.748434   0.640911  0.721791  0.596564  0.788930  0.576498   
Joseph       0.739581   0.671847  0.719362  0.592936  0.786319  0.582245   
Richard      0.730502   0.664337  0.725620  0.630241  0.779778  0.587989   
Charles      0.717849   0.691632  0.729507  0.648721  0.775796  0.581864   
Thomas       0.741932   0.663854  0.741360  0.625552  0.804401  0.576369   
Christopher  0.756968   0.651689  0.733535  0.589201  0.806338  0.591051   
Daniel       0.776290   0.650031 

In [7]:
# Female Names
# Pleasant Words
similarities_FvP = cosine_similarity(Female_Embeddings, Pleasant_Embeddings)
# Unpleasant Words
similarities_FvU = cosine_similarity(Female_Embeddings, Unpleasant_Embeddings)

similarities_FvP = pd.DataFrame(similarities_FvP, index = Female_Names, columns = Pleasant_Words)
similarities_FvU = pd.DataFrame(similarities_FvU, index = Female_Names, columns = Unpleasant_Words)

print("Cosine Similarity Matrix: Female Names vs Pleasant Words")
print(similarities_FvP)
print("Cosine Similarity Matrix: Female Names vs Unpleasant Words")
print(similarities_FvU)

Cosine Similarity Matrix: Female Names vs Pleasant Words
              happy  agreeable    polite     civil  charming  gracious  \
Mary       0.761429   0.654816  0.748546  0.597887  0.818544  0.638292   
Elizabeth  0.703669   0.649814  0.714691  0.629124  0.750184  0.584111   
Patricia   0.745577   0.687538  0.763339  0.640642  0.800270  0.603197   
Jennifer   0.781566   0.640885  0.747535  0.578465  0.831570  0.591445   
Linda      0.756707   0.688497  0.746467  0.638816  0.804329  0.583984   
Barbara    0.713708   0.657615  0.713795  0.634164  0.750501  0.575018   
Margaret   0.649873   0.652150  0.679634  0.643850  0.677559  0.565292   
Susan      0.774418   0.661733  0.754991  0.602863  0.817011  0.587846   
Dorothy    0.727479   0.654934  0.717493  0.646616  0.759422  0.569782   
Sarah      0.791057   0.652336  0.747058  0.596671  0.834395  0.620116   
Jessica    0.788571   0.633520  0.735989  0.565357  0.842707  0.585603   
Helen      0.778523   0.653650  0.740755  0.587707  0.8

# TEST 3: Gender Biases in Careers

In [8]:
# Male Names
# STEM Careers
similarities_MvS = cosine_similarity(Male_Embeddings, STEM_Embeddings)
# Non-STEM Careers
similarities_MvN = cosine_similarity(Male_Embeddings, Non_STEM_Embeddings)

similarities_MvS = pd.DataFrame(similarities_MvS, index = Male_Names, columns = STEM_Careers)
similarities_MvN = pd.DataFrame(similarities_MvN, index = Male_Names, columns = Non_STEM_Careers)

print("Cosine Similarity Matrix: Male Names vs STEM Careers")
print(similarities_MvS)
print("Cosine Similarity Matrix: Male Names vs Non-STEM Careers")
print(similarities_MvN)

Cosine Similarity Matrix: Male Names vs STEM Careers
             Software Developer  Nurse Practitioner  Health Services Manager  \
James                  0.599429            0.564889                 0.541916   
John                   0.591751            0.582338                 0.551145   
Robert                 0.613324            0.569287                 0.535666   
Michael                0.619155            0.585912                 0.545181   
William                0.588663            0.569451                 0.526834   
David                  0.576887            0.540213                 0.548638   
Joseph                 0.637130            0.593043                 0.582994   
Richard                0.582995            0.548710                 0.545192   
Charles                0.624249            0.606427                 0.541633   
Thomas                 0.610949            0.581891                 0.546688   
Christopher            0.617820            0.571468                

In [9]:
# Female Names
# STEM Careers
similarities_FvS = cosine_similarity(Female_Embeddings, STEM_Embeddings)
# Non-STEM Careers
similarities_FvN = cosine_similarity(Female_Embeddings, Non_STEM_Embeddings)

similarities_FvS = pd.DataFrame(similarities_FvS, index = Female_Names, columns = STEM_Careers)
similarities_FvN = pd.DataFrame(similarities_FvN, index = Female_Names, columns = Non_STEM_Careers)

print("Cosine Similarity Matrix: Female Names vs STEM Careers")
print(similarities_FvS)
print("Cosine Similarity Matrix: Female Names vs Non-STEM Careers")
print(similarities_FvN)

Cosine Similarity Matrix: Female Names vs STEM Careers
           Software Developer  Nurse Practitioner  Health Services Manager  \
Mary                 0.568754            0.563205                 0.527756   
Elizabeth            0.577916            0.594637                 0.533104   
Patricia             0.623632            0.629223                 0.564501   
Jennifer             0.595708            0.548935                 0.553280   
Linda                0.623720            0.629222                 0.553270   
Barbara              0.609831            0.626205                 0.558687   
Margaret             0.572103            0.617077                 0.522378   
Susan                0.598048            0.564146                 0.531240   
Dorothy              0.592929            0.610545                 0.555292   
Sarah                0.587028            0.561252                 0.525294   
Jessica              0.566091            0.518740                 0.535720   
Helen    

In [13]:
#Mean cosine similarity of each test

dataframes_dict = {
    'AFvP': similarities_AFvP,
    'AFvU': similarities_AFvU,
    'EUvP': similarities_EUvP,
    'EUvU': similarities_EUvU,
    'LXvP': similarities_LXvP,
    'LXvU': similarities_LXvU,
    'CHvP': similarities_CHvP,
    'CHvU': similarities_CHvU,
    'MvP': similarities_MvP,
    'MvU': similarities_MvU,
    'FvP': similarities_FvP,
    'FvU': similarities_FvU,
    'MvS': similarities_MvS,
    'MvN': similarities_MvN,
    'FvS': similarities_FvS,
    'FvN': similarities_FvN
}

# Create a dictionary to store the means
mean_dict = {}

# Calculate the mean for each DataFrame and store it in the mean_dict
for df_name, df in dataframes_dict.items():
    mean_value = df.values.mean()
    mean_dict[df_name] = mean_value

# Create a new DataFrame from the mean_dict
mean_df = pd.DataFrame(list(mean_dict.items()), columns=['DataFrame', 'avgCS_BERT_base_cased'])

# Print the new DataFrame
print(mean_df)

#Save to .csv
mean_df.to_csv('Dataframes/BERT_base_uncased_meanCosSim.csv', index = False)

   DataFrame  avgCS_BERT_base_cased
0       AFvP               0.595057
1       AFvU               0.631357
2       EUvP               0.709400
3       EUvU               0.754378
4       LXvP               0.713615
5       LXvU               0.761040
6       CHvP               0.626704
7       CHvU               0.673405
8        MvP               0.708827
9        MvU               0.755492
10       FvP               0.704018
11       FvU               0.749382
12       MvS               0.572057
13       MvN               0.683571
14       FvS               0.560526
15       FvN               0.678699


In [12]:
for key, df in dataframes_dict.items():
    # Construct the file path using the key
    file_path = f"Dataframes/{key}.csv"
    
    # Write the DataFrame to the CSV file
    df.to_csv(file_path, index=True)
    print(f"{key}.csv was created")

AFvP.csv was created
AFvU.csv was created
EUvP.csv was created
EUvU.csv was created
LXvP.csv was created
LXvU.csv was created
CHvP.csv was created
CHvU.csv was created
MvP.csv was created
MvU.csv was created
FvP.csv was created
FvU.csv was created
MvS.csv was created
MvN.csv was created
FvS.csv was created
FvN.csv was created
